# Asyncio Demo

In [1]:
import asyncio
from colorama import Fore

## Async Function

In [2]:
async def my_async_function(time):
    print('hello')
    await asyncio.sleep(time)
    print('world')

Wait for two seconds

In [7]:
await my_async_function(2)

hello
world


## Coroutines

In [4]:
my_async_function(2)

<coroutine object my_async_function at 0x000001304E3902C8>

### Running a Coroutine

Normally we would use `asyncio.run(<coroutine>)`

In [5]:
asyncio.run(my_async_function(2))

RuntimeError: asyncio.run() cannot be called from a running event loop

This doesn't work in Jupyter. Have to use the `await <coroutine>` syntax instead

In [8]:
await my_async_function(2)

hello
world


### Running multiple Coroutines

We can use the `asyncio.gather()` function to run multiple coroutines concurrently.

In [7]:
async def my_async_function(time,color):
    print(color + 'hello')
    await asyncio.sleep(time)
    print(color + 'world. Waited %.0f seconds' %time)

In [8]:
await asyncio.gather(
    my_async_function(2,Fore.BLUE),
    my_async_function(3,Fore.GREEN),
    my_async_function(4,Fore.RED),
    my_async_function(5,Fore.YELLOW)
);

hello
hello
hello
hello
world. Waited 2 seconds
world. Waited 3 seconds
world. Waited 4 seconds
world. Waited 5 seconds


In [9]:
async def another_async_function(loops,color):
    for i in range(0,loops):
        print(color + 'Other process loop %d' %(i+1))
        await asyncio.sleep(1)

In [10]:
await asyncio.gather(
    my_async_function(2,Fore.BLUE),
    my_async_function(3,Fore.BLUE),
    my_async_function(4,Fore.BLUE),
    my_async_function(5,Fore.BLUE),
    another_async_function(5,Fore.GREEN)
);

hello
hello
hello
hello
Other process loop 1
Other process loop 2
world. Waited 2 seconds
Other process loop 3
world. Waited 3 seconds
Other process loop 4
world. Waited 4 seconds
Other process loop 5
world. Waited 5 seconds


## Tasks

Run an unblocked group of coroutines

In [11]:
async def run_coroutines():
    await asyncio.gather(
        my_async_function(2,Fore.BLUE),
        my_async_function(3,Fore.BLUE),
        my_async_function(4,Fore.BLUE),
        my_async_function(5,Fore.BLUE),
        another_async_function(5,Fore.GREEN)
    );

In [12]:
task = asyncio.create_task(run_coroutines())
await asyncio.sleep(2)
print(Fore.RED + 'unblocked process')
await asyncio.sleep(2)
print(Fore.RED + 'still running')

hello
hello
hello
hello
Other process loop 1
Other process loop 2
unblocked process
world. Waited 2 seconds
Other process loop 3
world. Waited 3 seconds
Other process loop 4
world. Waited 4 seconds
still running
Other process loop 5
world. Waited 5 seconds


Stop a running task

In [13]:
global stop
stop = False
async def while_loop(color):
    global stop
    while (not stop):
        print(color + 'Still Running')
        await asyncio.sleep(1)
    print(color + 'Completed')

In [14]:
task = asyncio.create_task(while_loop(Fore.BLUE))

Still Running
Still Running
Still Running
Still Running
Still Running
Still Running
Still Running
Still Running
Still Running


In [15]:
print(Fore.RED + 'hi')

hi
Still Running
Still Running
Still Running
Still Running
Still Running
Still Running
Still Running
Still Running
Still Running


In [16]:
stop = True

Still Running
Completed


## Queues

Let's define a global queues that will be shared between coroutines

In [17]:
global my_queue1
global my_queue2
my_queue1 = asyncio.Queue()
my_queue2 = asyncio.Queue()

In [18]:
async def user1():
    global my_queue1
    global my_queue2
    print(Fore.BLUE + 'Sending message to user 2')
    await asyncio.sleep(3)
    await my_queue1.put('Hello user 2!')
    
    print(Fore.BLUE + 'Message sent. Waiting for response')
    user2_response = await my_queue2.get()
    print(Fore.BLUE + 'Message from user 2: ' + user2_response)
    
async def user2():
    global my_queue1
    global my_queue2
    print(Fore.RED + 'Waiting for message from user 1')
    user1_message = await my_queue1.get()
    
    print(Fore.RED + 'Message from user 1: ' + user1_message)
    print(Fore.RED + 'Sending response')
    await asyncio.sleep(3)
    await my_queue2.put('Hi, how are you user 1?')
    

In [19]:
asyncio.gather(
    user1(),
    user2()
);

Sending message to user 2
Waiting for message from user 1
Message sent. Waiting for response
Message from user 1: Hello user 2!
Sending response
Message from user 2: Hi, how are you user 1?
